In [1]:
from utils import *
import pandas as pd 
import numpy as np 

In [2]:
def get_resetter_params(row):
    if "_" in row:
        return float(row.split("_")[-1])
    else:
        return

In [3]:
df = pd.read_csv("../reports/adaptive_schedules/v7.csv.zip")
df = df[~df['resetter'].isin([ 'P-KSWIN_1', 'P-KSWIN_0.5',
       'P-KSWIN_0.125', 'P-KSWIN_0.0625'])] 
df['resetter'] = df['resetter'].replace('P-KSWIN_0.5', 'P-KSWIN')
df["accuracy"] *= 100

In [4]:
df.dataset.unique()

array(['Insects incr.', 'Insects gradual', 'RBF abrupt', 'RBF incr.',
       'SEA', 'Insects abrupt', 'Electricity', 'Covertype'], dtype=object)

In [5]:
# df = df[(df["resetter"] != "P-KSWIN") | (df["reset_lr"] == 0.5)]
df = df[( df["gamma"] == 1 - 2**-12 ) | (df['resetter'] == 'Kuncheva')]
df = df[["dataset", "resetter", "accuracy", 'base_lr', 'seed']]

In [6]:
# Define which data to show in table
datasets = [
    "RBF abrupt",
    "RBF incr.",
    "Covertype",
    "Electricity",
    "Insects abrupt",
    "Insects gradual",
    "Insects incr.",
]
ks = [1, 3]
columns = [(dataset, "accuracy") for dataset in datasets]
for k in ks:
    df = df[df["dataset"].isin(datasets)]
    table = get_topk_results(
        df,
        other_variables=["dataset", "resetter"],
        top_variable="base_lr",
        k=k,
    )
    table = table[["dataset", "resetter", "accuracy_mean", "accuracy_std"]]
    table = col_to_header(table, "dataset", index=["resetter"])
    mask_best = get_best_within_std(
        table, params_fixed=[], metrics=columns, modes="max"
    )
    table = merge_mean_std(table, columns, formats="{:.2f}")
    style = table.style
    style = apply_style_attribute(style, mask=mask_best)
    style.to_latex(f"../pub/tables/lr_resetting_top{k}.tex", hrules=True)

In [7]:
style

,RBF abrupt,RBF incr.,Covertype,Electricity,Insects abrupt,Insects gradual,Insects incr.
,accuracy,accuracy,accuracy,accuracy,accuracy,accuracy,accuracy
resetter,,,,,,,
ADWIN,94.28±.37,69.76±.92,82.70±.27,73.05±.45,71.27±.14,74.75±.21,60.26±.10
ADWIN Weight Reset,66.24±.55,61.12±.38,81.71±.06,69.06±.90,47.87±.34,58.95±1.10,30.51±.19
Ground Truth,94.58±.27,nan±nan,nan±nan,nan±nan,71.43±.13,nan±nan,nan±nan
KSWIN,94.23±.55,70.10±1.98,83.01±.06,73.13±.31,71.38±.16,74.71±.20,60.62±.19
Kuncheva,68.51±4.28,42.19±.64,76.96±.08,67.04±.02,66.63±.31,71.91±.35,52.07±.04
OneTailedADWIN,94.25±.38,70.22±2.58,82.64±.20,73.00±.50,71.25±.15,74.76±.14,60.28±.11
P-KSWIN_0.25,93.86±.48,70.71±1.38,83.01±.18,73.26±.43,71.25±.15,74.48±.25,59.95±.10
